In [137]:
import collections

In [77]:
import sys
import numpy as np

from astropy import units as u
from astropy import table, modeling

from scipy import stats

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import __version__ as mpl_vers
mpl_vers

'3.0.0'

In [3]:
pathtoinsert = '..'  # should have emceemr
if pathtoinsert not in sys.path:
    sys.path.insert(1, pathtoinsert)

import emceemr

In [4]:
import inspect

In [5]:
inspect.getmro(modeling.models.Gaussian1D)

(<class 'astropy.modeling.functional_models.Gaussian1D'>
Name: Gaussian1D
Inputs: ('x',)
Outputs: ('y',)
Fittable parameters: ('amplitude', 'mean', 'stddev'),
 <class 'astropy.modeling.core.Fittable1DModel'>,
 <class 'astropy.modeling.core.FittableModel'>,
 <class 'astropy.modeling.core.Model'>,
 object)

In [61]:
class ProbabilisticModel(modeling.Model):
    pass

class Test1(ProbabilisticModel, modeling.models.Gaussian1D):
    _inherit_descriptors_ = (modeling.Parameter,)
    proby = modeling.Parameter(default=0.5)
    
class Test2(modeling.models.Gaussian1D, ProbabilisticModel):
    _inherit_descriptors_ = (modeling.Parameter,)
    proby = modeling.Parameter(default=0.5)

Test1._parameters_, Test2._parameters_

(OrderedDict([('proby', Parameter('proby', default=0.5))]),
 OrderedDict([('amplitude', Parameter('amplitude', default=1)),
              ('mean', Parameter('mean', default=0)),
              ('stddev',
               Parameter('stddev', default=1, bounds=(1.1754943508222875e-38, None))),
              ('proby', Parameter('proby', default=0.5))]))

In [62]:
class ProbabilisticModel(modeling.Model):
    _inherit_descriptors_ = (modeling.Parameter,)

class Test1(ProbabilisticModel, modeling.models.Gaussian1D):
    proby = modeling.Parameter(default=0.5)
    
class Test2(modeling.models.Gaussian1D, ProbabilisticModel):
    proby = modeling.Parameter(default=0.5)

Test1._parameters_, Test2._parameters_

(OrderedDict([('amplitude', Parameter('amplitude', default=1)),
              ('mean', Parameter('mean', default=0)),
              ('stddev',
               Parameter('stddev', default=1, bounds=(1.1754943508222875e-38, None))),
              ('proby', Parameter('proby', default=0.5))]),
 OrderedDict([('amplitude', Parameter('amplitude', default=1)),
              ('mean', Parameter('mean', default=0)),
              ('stddev',
               Parameter('stddev', default=1, bounds=(1.1754943508222875e-38, None))),
              ('proby', Parameter('proby', default=0.5))]))

In [63]:
class ProbabilisticModel(modeling.Model):
    _inherit_descriptors_ = (modeling.Parameter,)
    
    
    

Base API: ProbabilisticModel should have a mean model and a noise model.

Noise Model should take the mean model + data and output a log-prob of the same shape as the mean model output.  Form can be one of
 * modeling model that returns log-prob given whole mean-model (one of the parameters is the mean-model part)?
 * modeling model that returns prob given whole mean-model (one of the parameters is the mean-model part)?
 * scipy.stats probability distribution. need to figure out how to get parameters...
 * callable that takes in log-prob
    
For experimentation, use a linear mean model and gaussian noise model

In [76]:
ndata = 100

mean_model = modeling.models.Linear1D(slope=2, intercept=1)

Need to make the log-prob model:

In [237]:
def generate_logpdfmodel_class(frozen_distribution, scale_param_name='prob_scale', location_param_name='prob_loc', data_param_name='prob_loc'):
    dist = frozen_distribution.dist
    other_params, loc, scale = dist._parse_args(*frozen_distribution.args, **frozen_distribution.kwds)
    other_param_names = [] if dist.shapes is None else dist.shapes.split(', ')

    assert len(other_params) == len(other_param_names), 'This stats object seems messed up - the names of the parameters do not match the number of parameters'

    # order matters here!  most go "other parameters in order, loc, scale"
    params = []
    for nm, p in zip(other_param_names, other_params):
        params.append(modeling.Parameter(name=nm, default=p))
    params.append(modeling.Parameter(name=location_param_name, default=loc, description='distribution location parameter'))
    params.append(modeling.Parameter(name=scale_param_name, default=scale, description='distribution scale parameter'))

    params = collections.OrderedDict(((p.name, p) for p in params))
    
    members = {}
    
    def tester(*args, **kwargs):
        print(args, kwargs)
        return dist.logpdf(*args, **kwargs)
        
    
    mod = find_current_module(1)
    modname = mod.__name__ if mod else '__main__'
    
    members = {
        '__module__': str(modname),
        'distribution': dist,
        'inputs': ('x',),
        'outputs': ('logpdf',),
        'evaluate': staticmethod(dist.logpdf),
    }
    members.update(params)
    
    return type(dist.name.title() + 'LogPDFModel', (modeling.FittableModel,), members)
    
    
Mod = generate_logpdfmodel_class(stats.beta(1,2, scale=3))
Mod

<class '__main__.BetaLogPDFModel'>
Name: BetaLogPDFModel
Inputs: ('x',)
Outputs: ('logpdf',)
Fittable parameters: ('a', 'b', 'prob_loc', 'prob_scale')

In [240]:
mod = Mod()
mod(1), stats.beta.logpdf(1, 1, 2, scale=3)

(-0.8109302162163288, -0.8109302162163288)

In [257]:
Mod2 = generate_logpdfmodel_class(stats.norm(scale=1), scale_param_name='sigma')
Mod2

<class '__main__.NormLogPDFModel'>
Name: NormLogPDFModel
Inputs: ('x',)
Outputs: ('logpdf',)
Fittable parameters: ('prob_loc', 'sigma')

In [258]:
mod = Mod2(prob_loc=[0, 1])
mod(0)

array([-0.91893853, -1.41893853])

Now try the core functionality

In [266]:
noise_model_class = generate_logpdfmodel_class(stats.norm(), location_param_name='mu', scale_param_name='sigma')
noise_model_class

<class '__main__.NormLogPDFModel'>
Name: NormLogPDFModel
Inputs: ('x',)
Outputs: ('logpdf',)
Fittable parameters: ('mu', 'sigma')

In [268]:
noise_model = noise_model_class()
noise_model

<NormLogPDFModel(mu=0., sigma=1.)>

In [272]:
combined_model(-.5)

-0.9189385332046727

That's not quite right because the model is going in as the *input* not the mean...

In [284]:
combined_model = (mean_model & modeling.models.Identity(1)) | noise_model
combined_model

ModelDefinitionError: Unsupported operands for |: None (n_inputs=2, n_outputs=2) and None (n_inputs=1, n_outputs=1); n_outputs for the left-hand model must match n_inputs for the right-hand model.

In [308]:
def generate_logpdfmodel_class(frozen_distribution,
                               scale_param_name='prob_scale',
                               location_param_name='prob_loc',
                               mean_model_param_name=None):
    """
    mean_model_param_name = None means "whatever the location parameter is"
    """
    if mean_model_param_name is None:
        mean_model_param_name = location_param_name

    dist = frozen_distribution.dist
    other_params, loc, scale = dist._parse_args(*frozen_distribution.args, **frozen_distribution.kwds)
    other_param_names = [] if dist.shapes is None else dist.shapes.split(', ')

    assert len(other_params) == len(other_param_names), 'This stats object seems messed up - the names of the parameters do not match the number of parameters'

    # order matters here!  most go "other parameters in order, loc, scale"
    params = []
    for nm, p in zip(other_param_names, other_params):
        params.append(modeling.Parameter(name=nm, default=p))
    params.append(modeling.Parameter(name=location_param_name, default=loc, description='distribution location parameter'))
    params.append(modeling.Parameter(name=scale_param_name, default=scale, description='distribution scale parameter'))

    params = collections.OrderedDict(((p.name, p) for p in params if p.name != mean_model_param_name))

    mod = find_current_module(2)
    modname = mod.__name__ if mod else '__main__'

    def log_pdf_with_meanmodel(*args, **kwargs):
        print('init args', args, '|', kwargs)
        kwargs[mean_model_param_name] = args[1]
        print('calling w', args, kwargs)
        return dist.logpdf(args[0], **kwargs)

    members = {
        '__module__': str(modname),
        'distribution': dist,
        'inputs': ('data', 'mean_model'),
        'outputs': ('logpdf',),
        'evaluate': staticmethod(log_pdf_with_meanmodel),
    }
    members.update(params)

    return type(dist.name.title() + 'LogPDFModel', (modeling.FittableModel,), members)


noise_model_class = generate_logpdfmodel_class(stats.norm(), location_param_name='mu', scale_param_name='sigma')
noise_model = noise_model_class()

combined_model = (modeling.models.Identity(1) & mean_model) | noise_model
combined_model

<CompoundModel15(slope_1=2., intercept_1=1., sigma_2=1.)>

In [312]:
noise_model.parameters

array([1.])

In [309]:
noise_model([3], [4])

init args (array([3.]), array([4.]), array([1.])) | {}
calling w (array([3.]), array([4.]), array([1.])) {'mu': array([4.])}


TypeError: _parse_args() got an unexpected keyword argument 'mu'

In [307]:
debug

> /Users/erik/miniconda3/envs/astro36/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py(1700)logpdf()
   1698 
   1699         """
-> 1700         args, loc, scale = self._parse_args(*args, **kwds)
   1701         x, loc, scale = map(asarray, (x, loc, scale))
   1702         args = tuple(map(asarray, args))



ipdb>  u


> <ipython-input-302-cae5ebe16bba>(32)log_pdf_with_meanmodel()
     30         kwargs[mean_model_param_name] = args[1]
     31         print('calling w', args, kwargs)
---> 32         return dist.logpdf(args[0], **kwargs)
     33 
     34     members = {



ipdb>  u


> /Users/erik/miniconda3/envs/astro36/lib/python3.6/site-packages/astropy/modeling/core.py(815)__call__()
    813                     outputs = [np.asarray(r) for r in result]
    814         else:
--> 815             outputs = self.evaluate(*chain(inputs, parameters))
    816         if self.n_outputs == 1:
    817             outputs = (outputs,)



...

ipdb>  p self.param_names


('sigma',)


ipdb>  d


> <ipython-input-302-cae5ebe16bba>(32)log_pdf_with_meanmodel()
     30         kwargs[mean_model_param_name] = args[1]
     31         print('calling w', args, kwargs)
---> 32         return dist.logpdf(args[0], **kwargs)
     33 
     34     members = {



ipdb>  exit


In [303]:
combined_model(1, 2)

calling w (array([1.]), array([5.]), array([1.])) {'mu': array([5.])}


TypeError: _parse_args() got an unexpected keyword argument 'mu'

<CompoundModel7(slope_0=2., intercept_0=1., sigma_2=1.)>